In [24]:
pip install boto3 botocore s3transfer

  Using cached boto3-1.34.145-py3-none-any.whl.metadata (6.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached botocore-1.34.145-py3-none-any.whl.metadata (5.7 kB)
Using cached boto3-1.34.145-py3-none-any.whl (139 kB)
Using cached botocore-1.34.145-py3-none-any.whl (12.4 MB)
Using cached s3transfer-0.10.2-py3-none-any.whl (82 kB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.29
    Uninstalling botocore-1.34.29:
      Successfully uninstalled botocore-1.34.29
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awsebcli 3.20.10 requires botocore<1.32.0,>1.23.41, but you have botocore 1.34.145 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import boto3
import pandas as pd
import io

# AWS S3 버킷 정보
bucket_name = 'papalio-test-bucket'
file_key = 'test_otto/products_with_size_color.csv'

# S3 클라이언트 생성
s3 = boto3.client('s3')

# S3에서 CSV 파일 다운로드
try:
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    content = response['Body'].read().decode('utf-8')
    
    # CSV 내용을 데이터프레임으로 변환
    df = pd.read_csv(io.StringIO(content))
    
    # 데이터프레임 출력
    #print(df)
    #df csv파일로 만들기
    
except Exception as e:
    print(f"Error reading S3 object: {e}")


In [26]:
import boto3
import pandas as pd
import io
import re
#production preprocessing
df['product_id'] = df.index.to_series().apply(lambda x: f'top_{x}')
# price 열의 데이터를 전처리
for i in range(len(df)):
    price_value = df.loc[i, 'price']
    # 빈칸을 기준으로 앞부분만 남기고 나머지 제거
    price_value = price_value.split(' ')[0]
    # 숫자와 쉼표만 남기기
    price_value = re.sub(r'[^\d,]', '', price_value)
    df.loc[i, 'price'] = price_value



In [27]:
import pandas as pd

# Assuming df is already loaded

colors = [
    "다크그레이", "다크네이비", "다크베이지", "라임", "로얄네이비", "브라운(품절)", "블랙", "아이보리", 
    "애쉬카키(품절)", "연그레이", "진카키(품절)", "크림", "레몬", "더스티오렌지", "터키블루", "다크그린", 
    "퍼플", "네이비", "라이트멜란지", "그레이", "그린", "잉크블루", "머스타드", "차콜", "블루그레이", 
    "구스그레이", "스톤그레이", "더스티민트", "멜란지", "애쉬블루", "화이트/화이트", "화이트/블랙", 
    "블랙/블랙", "BROWN", "블랙(품절)", "블루(품절)", "라이트 카키(품절)", "라이트 그레이(품절)", 
    "화이트(품절)", "퍼플", "올리브", "핑크", "옐로우", "민트", "오렌지", "마운틴그린", "백오트밀", 
    "크림", "틸블루", "페일퍼플", "Xsmall",
    "모카 베이지", "라임", "로얄네이비", "브라운", "아이보리", "애쉬카키", "연그레이", "진카키", "크림", 
    "레몬", "더스티오렌지", "터키블루", "다크그린", "퍼플", "네이비", "라이트멜란지", "그레이", "그린", 
    "잉크블루", "머스타드", "차콜", "블루그레이", "구스그레이", "스톤그레이", "더스티민트", "멜란지", 
    "애쉬블루", "화이트/화이트", "화이트/블랙", "블랙/블랙", "BROWN", "블랙", "블루", "라이트 카키", 
    "라이트 그레이", "화이트", "올리브", "핑크", "옐로우", "민트", "오렌지", "마운틴그린", "백오트밀", 
    "크림", "틸블루", "페일퍼플", "Xsmall"
]

def contains_color_option(size_options, colors):
    for color in colors:
        if color in size_options:
            return True
    return False

for i in range(len(df)):
    if contains_color_option(df.loc[i, 'size_options'], colors):
        # Swap size_options and color_options
        df.loc[i, 'size_options'], df.loc[i, 'color_options'] = df.loc[i, 'color_options'], df.loc[i, 'size_options']
        
#df = df.drop('size_options',axis=1)

In [24]:
df.to_csv("preprocessing_test.csv", index=False, encoding='utf-8-sig')